In [44]:
import pandas as pd
import numpy as np
import datetime
import importlib
import pickle
import time

import data_utils
import model_utils
import train_utils
import evaluation
import submission
importlib.reload(data_utils)
importlib.reload(model_utils)
importlib.reload(train_utils)
importlib.reload(evaluation)
importlib.reload(submission)

import torch
import torch.nn as nn

### Training 

In [45]:
file_dict = {"train": "./data/train_real.csv", 
             "val_text": "./hw2_utterance_dev.txt",
             "val_label": "./hw2_tags_dev.txt",
             "test_text": "./hw2_utterance_test.txt",
             "holdout": "./data/holdout.csv"}
train_data, val_data, test_data, info = data_utils.prep_all_data(
    file_dict, batch_size=1, save=True)

In [25]:
importlib.reload(model_utils)
m = model_utils.GRU(info, direction=2).cuda()
result = train_utils.train(train_data, val_data, m, lr=1e-3, patience=10)

Epoch: 0, LR: 0.001, Train Loss: 151.5429, Val Loss: 88.4779, Val f1 0.600, epoch time: 2.7s
Epoch: 1, LR: 0.001, Train Loss: 47.2335, Val Loss: 76.9151, Val f1 0.683, epoch time: 2.8s
Epoch: 2, LR: 0.001, Train Loss: 22.1414, Val Loss: 62.3697, Val f1 0.738, epoch time: 2.8s
Epoch: 3, LR: 0.001, Train Loss: 11.0858, Val Loss: 65.0368, Val f1 0.757, epoch time: 2.8s
Epoch: 4, LR: 0.001, Train Loss: 5.6552, Val Loss: 67.8574, Val f1 0.768, epoch time: 2.8s
Epoch: 5, LR: 0.001, Train Loss: 3.2823, Val Loss: 64.2119, Val f1 0.761, epoch time: 2.8s
Epoch: 6, LR: 0.001, Train Loss: 2.2067, Val Loss: 67.2561, Val f1 0.758, epoch time: 2.7s
Epoch: 7, LR: 0.001, Train Loss: 1.4807, Val Loss: 72.0254, Val f1 0.757, epoch time: 2.8s
Epoch: 8, LR: 0.001, Train Loss: 1.1186, Val Loss: 70.5598, Val f1 0.767, epoch time: 2.7s
Epoch: 9, LR: 0.001, Train Loss: 0.8599, Val Loss: 69.2148, Val f1 0.775, epoch time: 2.8s
Epoch: 10, LR: 0.001, Train Loss: 0.7789, Val Loss: 70.5466, Val f1 0.775, epoch time

In [10]:
m_GRU = result["trained_model"]

In [5]:
importlib.reload(model_utils)
importlib.reload(train_utils)
importlib.reload(data_utils)

train_data, val_data, info = data_utils.prep_all_data(
    file_dict, batch_size=1)
m = model_utils.BertSlot(info).cuda()
result = train_utils.train(train_data, val_data, m, lr=1e-5, patience=5)

Epoch: 0, LR: 1e-05, Train Loss: 6.9148, Val Loss: 5.9229, Val f1 0.130, epoch time: 123.4s
Epoch: 1, LR: 1e-05, Train Loss: 4.6921, Val Loss: 5.0505, Val f1 0.164, epoch time: 124.3s
Epoch: 2, LR: 1e-05, Train Loss: 3.6306, Val Loss: 4.2201, Val f1 0.353, epoch time: 125.2s
Epoch: 3, LR: 1e-05, Train Loss: 2.8912, Val Loss: 3.7763, Val f1 0.482, epoch time: 124.3s
Epoch: 4, LR: 1e-05, Train Loss: 2.4406, Val Loss: 3.5336, Val f1 0.582, epoch time: 124.0s
Epoch: 5, LR: 1e-05, Train Loss: 2.0189, Val Loss: 3.7444, Val f1 0.467, epoch time: 124.5s
Epoch: 6, LR: 1e-05, Train Loss: 1.7790, Val Loss: 3.4687, Val f1 0.584, epoch time: 124.7s
Epoch: 7, LR: 1e-05, Train Loss: 1.5135, Val Loss: 3.1951, Val f1 0.673, epoch time: 125.6s
Epoch: 8, LR: 1e-05, Train Loss: 1.2945, Val Loss: 3.2953, Val f1 0.705, epoch time: 123.3s
Epoch: 9, LR: 1e-05, Train Loss: 1.2025, Val Loss: 3.0144, Val f1 0.689, epoch time: 125.7s
Epoch: 10, LR: 1e-05, Train Loss: 1.0526, Val Loss: 2.5923, Val f1 0.769, epoch 

In [16]:
torch.save(result["trained_model"].state_dict(), "./data/model_checkpoints/bertSlot_Feb28.mdl")

In [17]:
m_bert = result["trained_model"]

In [32]:
importlib.reload(model_utils)
m = model_utils.BaseModel(info, pretrain="ft").cuda()
train_data, val_data, info = data_utils.prep_all_data(file_dict, batch_size=1)
result = train_utils.train(train_data, val_data, m, lr=1e-3, patience=5)

Epoch: 0, LR: 0.001, Train Loss: 4.8025, Val Loss: 6.3148, Val f1 0.616, epoch time: 4.1s
Epoch: 1, LR: 0.001, Train Loss: 1.9605, Val Loss: 5.9936, Val f1 0.594, epoch time: 4.1s
Epoch: 2, LR: 0.001, Train Loss: 1.5960, Val Loss: 5.8579, Val f1 0.616, epoch time: 4.1s
Epoch: 3, LR: 0.001, Train Loss: 1.4659, Val Loss: 5.4492, Val f1 0.607, epoch time: 4.0s
Epoch: 4, LR: 0.001, Train Loss: 1.4150, Val Loss: 5.4308, Val f1 0.586, epoch time: 4.0s
Epoch: 5, LR: 0.001, Train Loss: 1.3831, Val Loss: 5.2612, Val f1 0.600, epoch time: 4.0s
Epoch: 6, LR: 0.001, Train Loss: 1.3727, Val Loss: 5.1516, Val f1 0.618, epoch time: 4.0s
Epoch: 7, LR: 0.001, Train Loss: 1.3678, Val Loss: 5.1899, Val f1 0.607, epoch time: 3.9s
Epoch: 8, LR: 0.001, Train Loss: 1.3534, Val Loss: 5.2619, Val f1 0.592, epoch time: 4.1s
Epoch: 9, LR: 0.001, Train Loss: 1.3527, Val Loss: 5.1682, Val f1 0.554, epoch time: 4.0s
Epoch: 10, LR: 0.001, Train Loss: 1.3490, Val Loss: 5.2218, Val f1 0.552, epoch time: 3.9s
Epoch: 11

In [33]:
importlib.reload(model_utils)
m = model_utils.BaseModel(info, pretrain="glove").cuda()
train_data, val_data, info = data_utils.prep_all_data(file_dict, batch_size=1)
result = train_utils.train(train_data, val_data, m, lr=1e-3, patience=5)

Epoch: 0, LR: 0.001, Train Loss: 3.8290, Val Loss: 4.4245, Val f1 0.532, epoch time: 3.9s
Epoch: 1, LR: 0.001, Train Loss: 1.9041, Val Loss: 4.6375, Val f1 0.546, epoch time: 4.0s
Epoch: 2, LR: 0.001, Train Loss: 1.5889, Val Loss: 4.8041, Val f1 0.550, epoch time: 4.1s
Epoch: 3, LR: 0.001, Train Loss: 1.4834, Val Loss: 5.0831, Val f1 0.550, epoch time: 4.0s
Epoch: 4, LR: 0.001, Train Loss: 1.4441, Val Loss: 5.1472, Val f1 0.542, epoch time: 4.0s
Epoch: 5, LR: 0.001, Train Loss: 1.4089, Val Loss: 5.3307, Val f1 0.530, epoch time: 4.1s
Epoch: 6, LR: 0.0001, Train Loss: 1.4039, Val Loss: 5.3643, Val f1 0.542, epoch time: 4.0s
Epoch: 7, LR: 0.0001, Train Loss: 1.2760, Val Loss: 5.3755, Val f1 0.545, epoch time: 4.0s
Epoch: 8, LR: 0.0001, Train Loss: 1.2476, Val Loss: 5.4066, Val f1 0.539, epoch time: 4.0s


In [34]:
importlib.reload(model_utils)
m = model_utils.BaseModel(info, pretrain=False).cuda()
train_data, val_data, info = data_utils.prep_all_data(file_dict, batch_size=1)
result = train_utils.train(train_data, val_data, m, lr=1e-3, patience=5)

Epoch: 0, LR: 0.001, Train Loss: 5.5819, Val Loss: 4.1666, Val f1 0.521, epoch time: 4.0s
Epoch: 1, LR: 0.001, Train Loss: 2.6193, Val Loss: 3.7894, Val f1 0.578, epoch time: 4.1s
Epoch: 2, LR: 0.001, Train Loss: 2.0214, Val Loss: 3.7392, Val f1 0.589, epoch time: 4.0s
Epoch: 3, LR: 0.001, Train Loss: 1.7922, Val Loss: 3.8118, Val f1 0.590, epoch time: 4.0s
Epoch: 4, LR: 0.001, Train Loss: 1.6759, Val Loss: 4.0622, Val f1 0.541, epoch time: 4.0s
Epoch: 5, LR: 0.001, Train Loss: 1.6106, Val Loss: 3.9454, Val f1 0.524, epoch time: 3.6s
Epoch: 6, LR: 0.001, Train Loss: 1.6115, Val Loss: 4.1069, Val f1 0.528, epoch time: 3.8s
Epoch: 7, LR: 0.001, Train Loss: 1.6057, Val Loss: 4.3533, Val f1 0.485, epoch time: 3.6s
Epoch: 8, LR: 0.0001, Train Loss: 1.5804, Val Loss: 4.3187, Val f1 0.510, epoch time: 3.9s
Epoch: 9, LR: 0.0001, Train Loss: 1.3332, Val Loss: 4.1779, Val f1 0.532, epoch time: 3.7s


In [16]:
importlib.reload(model_utils)
m = model_utils.BaseModel(info).cuda()
train_data, val_data, info = data_utils.prep_all_data(file_dict, batch_size=1)
result = train_utils.train(train_data, val_data, m, lr=1e-3, patience=5)

Epoch: 0, LR: 0.001, Train Loss: 5.6728, Val Loss: 4.4029, Val f1 0.514, epoch time: 4.0s
Epoch: 1, LR: 0.001, Train Loss: 2.6445, Val Loss: 3.8269, Val f1 0.569, epoch time: 3.9s
Epoch: 2, LR: 0.001, Train Loss: 2.0626, Val Loss: 3.8600, Val f1 0.547, epoch time: 4.0s
Epoch: 3, LR: 0.001, Train Loss: 1.8022, Val Loss: 3.9125, Val f1 0.581, epoch time: 4.0s
Epoch: 4, LR: 0.001, Train Loss: 1.6839, Val Loss: 4.0605, Val f1 0.578, epoch time: 3.9s
Epoch: 5, LR: 0.001, Train Loss: 1.6443, Val Loss: 4.2622, Val f1 0.554, epoch time: 3.1s
Epoch: 6, LR: 0.001, Train Loss: 1.6232, Val Loss: 4.3333, Val f1 0.526, epoch time: 3.7s
Epoch: 7, LR: 0.0001, Train Loss: 1.5871, Val Loss: 4.3168, Val f1 0.573, epoch time: 3.6s
Epoch: 8, LR: 0.0001, Train Loss: 1.3460, Val Loss: 4.2309, Val f1 0.567, epoch time: 3.6s
Epoch: 9, LR: 0.0001, Train Loss: 1.2810, Val Loss: 4.2417, Val f1 0.578, epoch time: 3.6s


In [17]:
result["total_time"]

38.588520765304565

In [18]:
importlib.reload(model_utils)
m = model_utils.BaseModel(info).cuda()
train_data, val_data, info = data_utils.prep_all_data(file_dict, batch_size=-1)
result = train_utils.train(train_data, val_data, m, lr=1e-3, patience=10)

Epoch: 0, LR: 0.001, Train Loss: 414.4476, Val Loss: 226.1604, Val f1 0.261, epoch time: 1.5s
Epoch: 1, LR: 0.001, Train Loss: 211.1787, Val Loss: 153.8800, Val f1 0.411, epoch time: 1.5s
Epoch: 2, LR: 0.001, Train Loss: 145.3856, Val Loss: 123.8750, Val f1 0.461, epoch time: 1.4s
Epoch: 3, LR: 0.001, Train Loss: 113.9929, Val Loss: 108.7808, Val f1 0.506, epoch time: 1.4s
Epoch: 4, LR: 0.001, Train Loss: 95.3859, Val Loss: 100.2948, Val f1 0.534, epoch time: 1.4s
Epoch: 5, LR: 0.001, Train Loss: 82.7822, Val Loss: 94.9902, Val f1 0.546, epoch time: 1.4s
Epoch: 6, LR: 0.001, Train Loss: 73.6048, Val Loss: 91.5024, Val f1 0.564, epoch time: 1.4s
Epoch: 7, LR: 0.001, Train Loss: 66.6497, Val Loss: 89.1499, Val f1 0.563, epoch time: 1.4s
Epoch: 8, LR: 0.001, Train Loss: 61.2569, Val Loss: 87.5647, Val f1 0.567, epoch time: 1.4s
Epoch: 9, LR: 0.001, Train Loss: 57.0154, Val Loss: 86.5397, Val f1 0.569, epoch time: 1.4s
Epoch: 10, LR: 0.001, Train Loss: 53.6381, Val Loss: 85.9299, Val f1 0.

In [19]:
result["total_time"]

48.02327919006348

In [ ]:
importlib.reload(model_utils)
m = model_utils.GRU(info, direction=1).cuda()
train_data, val_data, info = data_utils.prep_all_data(file_dict, batch_size=1)
result = train_utils.train(train_data, val_data, m, lr=1e-3, patience=5)

In [24]:
result["total_time"]

139.14653182029724

In [25]:
importlib.reload(model_utils)
m = model_utils.GRU(info, direction=1).cuda()
train_data, val_data, info = data_utils.prep_all_data(file_dict, batch_size=-1)
result = train_utils.train(train_data, val_data, m, lr=1e-3, patience=10)

Epoch: 0, LR: 0.001, Train Loss: 193.9474, Val Loss: 114.9780, Val f1 0.382, epoch time: 2.1s
Epoch: 1, LR: 0.001, Train Loss: 75.5452, Val Loss: 83.8583, Val f1 0.617, epoch time: 2.1s
Epoch: 2, LR: 0.001, Train Loss: 43.8587, Val Loss: 68.0680, Val f1 0.698, epoch time: 2.1s
Epoch: 3, LR: 0.001, Train Loss: 28.7550, Val Loss: 58.0142, Val f1 0.708, epoch time: 2.1s
Epoch: 4, LR: 0.001, Train Loss: 19.8479, Val Loss: 55.5259, Val f1 0.721, epoch time: 2.1s
Epoch: 5, LR: 0.001, Train Loss: 14.6029, Val Loss: 56.5672, Val f1 0.726, epoch time: 2.1s
Epoch: 6, LR: 0.001, Train Loss: 11.5257, Val Loss: 58.4251, Val f1 0.726, epoch time: 2.1s
Epoch: 7, LR: 0.001, Train Loss: 9.6457, Val Loss: 58.4262, Val f1 0.729, epoch time: 2.1s
Epoch: 8, LR: 0.001, Train Loss: 8.2930, Val Loss: 57.4351, Val f1 0.733, epoch time: 2.1s
Epoch: 9, LR: 0.001, Train Loss: 7.0162, Val Loss: 57.2999, Val f1 0.738, epoch time: 2.1s
Epoch: 10, LR: 0.001, Train Loss: 5.8338, Val Loss: 58.0442, Val f1 0.746, epoch 

In [26]:
result["total_time"]

47.34501576423645

In [30]:
importlib.reload(model_utils)
m = model_utils.GRU(info, direction=2).cuda()
train_data, val_data, info = data_utils.prep_all_data(file_dict, batch_size=1)
result = train_utils.train(train_data, val_data, m, lr=1e-3, patience=5)
print(result["total_time"])

Epoch: 0, LR: 0.001, Train Loss: 2.3658, Val Loss: 2.0394, Val f1 0.753, epoch time: 20.6s
Epoch: 1, LR: 0.001, Train Loss: 0.4229, Val Loss: 2.1754, Val f1 0.770, epoch time: 22.8s
Epoch: 2, LR: 0.001, Train Loss: 0.2173, Val Loss: 2.4627, Val f1 0.790, epoch time: 22.7s
Epoch: 3, LR: 0.001, Train Loss: 0.1122, Val Loss: 2.6050, Val f1 0.791, epoch time: 22.6s
Epoch: 4, LR: 0.001, Train Loss: 0.0813, Val Loss: 2.8167, Val f1 0.778, epoch time: 23.2s
Epoch: 5, LR: 0.001, Train Loss: 0.0774, Val Loss: 3.1432, Val f1 0.762, epoch time: 23.1s
Epoch: 6, LR: 0.0001, Train Loss: 0.0470, Val Loss: 3.1717, Val f1 0.778, epoch time: 23.4s
Epoch: 7, LR: 0.0001, Train Loss: 0.0304, Val Loss: 3.1086, Val f1 0.785, epoch time: 23.0s
Epoch: 8, LR: 0.0001, Train Loss: 0.0099, Val Loss: 3.1131, Val f1 0.787, epoch time: 23.4s
Epoch: 9, LR: 0.0001, Train Loss: 0.0082, Val Loss: 3.1147, Val f1 0.787, epoch time: 23.3s
232.24305272102356


In [3]:
importlib.reload(model_utils)
m = model_utils.GRU(info, direction=2).cuda()
result = train_utils.train(train_data, val_data, m)

Epoch: 0, LR: 0.001, Train Loss: 2.4104, Val Loss: 2.6255, Val f1 0.714
Epoch: 1, LR: 0.001, Train Loss: 0.4522, Val Loss: 2.1465, Val f1 0.764
Epoch: 2, LR: 0.001, Train Loss: 0.1908, Val Loss: 2.1976, Val f1 0.787
Epoch: 3, LR: 0.001, Train Loss: 0.1224, Val Loss: 2.3299, Val f1 0.791
Epoch: 4, LR: 0.001, Train Loss: 0.0878, Val Loss: 2.7668, Val f1 0.786
Epoch: 5, LR: 0.001, Train Loss: 0.0662, Val Loss: 2.8125, Val f1 0.805
Epoch: 6, LR: 0.001, Train Loss: 0.0486, Val Loss: 2.8417, Val f1 0.808
Epoch: 7, LR: 0.001, Train Loss: 0.0381, Val Loss: 3.0598, Val f1 0.793
Epoch: 8, LR: 0.001, Train Loss: 0.0446, Val Loss: 3.0771, Val f1 0.799
Epoch: 9, LR: 0.001, Train Loss: 0.0297, Val Loss: 3.2020, Val f1 0.805
Epoch: 10, LR: 0.001, Train Loss: 0.0285, Val Loss: 3.3087, Val f1 0.796
Epoch: 11, LR: 0.001, Train Loss: 0.0511, Val Loss: 3.7601, Val f1 0.774
Epoch: 12, LR: 0.0001, Train Loss: 0.0183, Val Loss: 4.0560, Val f1 0.782
Epoch: 13, LR: 0.0001, Train Loss: 0.0187, Val Loss: 4.0025,

In [8]:
torch.save(result["trained_model"], "./data/model_checkpoints/biGRU_Feb23.mdl")

### Generate submission file

In [46]:
importlib.reload(submission)
submission.get_submission(model=m_bert, data=test_data)

prediction file saved to: prediction.txt
